In [1]:
!pip install yfinance
!git clone https://github.com/amazon-science/chronos-forecasting.git
%cd chronos-forecasting
!pip install -e .[training]

Cloning into 'chronos-forecasting'...
remote: Enumerating objects: 449, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (142/142), done.
remote: Total 449 (delta 168), reused 86 (delta 86), pack-reused 221 (from 2)
Receiving objects: 100% (449/449), 1008.56 KiB | 3.15 MiB/s, done.
Resolving deltas: 100% (188/188), done.
/content/chronos-forecasting
Obtaining file:///content/chronos-forecasting
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 28.2 MB/s eta 0:00:00
 

In [2]:
import yfinance as yf

tickers = ["AAPL"]
time_series_list = []

for ticker in tickers:
    df = yf.download(ticker, start="2013-01-01", end="2023-01-01")
    time_series_list.append(df["Close"].values.flatten())

[*********************100%***********************]  1 of 1 completed


In [3]:
from pathlib import Path
from typing import List, Union
import numpy as np
from gluonts.dataset.arrow import ArrowWriter

def convert_to_arrow(
    path: Union[str, Path],
    time_series: Union[List[np.ndarray], np.ndarray],
    compression: str = "lz4",
):
    """
    Salva un insieme di serie temporali in formato Arrow nel percorso specificato.
    Ogni serie viene appiattita per essere garantita come array 1D.
    """
    # Impostiamo un tempo di inizio arbitrario
    start = np.datetime64("2000-01-01 00:00", "s")

    # Per ogni serie, assicuriamoci che sia un array 1D
    dataset = [{"start": start, "target": np.array(ts).flatten()} for ts in time_series]

    ArrowWriter(compression=compression).write_to_file(dataset, path=path)
    print(f"File Arrow salvato in: {path}")

# Salva i dati in /content/stock_data.arrow
convert_to_arrow("/content/stock_data.arrow", time_series=time_series_list)

File Arrow salvato in: /content/stock_data.arrow


In [5]:
%%writefile /content/config.yaml
training_data_paths:
  - "/content/stock_data.arrow"
probability:
  - 1.0

model_id: "amazon/chronos-t5-tiny"
random_init: false

max_steps: 1000
learning_rate: 0.001

shuffle_buffer_length: 1000
torch_compile: false

Overwriting /content/config.yaml


In [6]:
!CUDA_VISIBLE_DEVICES=0 python scripts/training/train.py --config /content/config.yaml

2025-02-01 17:03:03.438817: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738429383.673345    1181 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738429383.738107    1181 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-01 17:03:04.227028: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-02-01 17:03:11,051 - /content/chronos-forecasting/scripts/training/train.py - INFO - TF32 format is only availab

In [7]:
!pip install "chronos-forecasting[evaluation] @ git+https://github.com/amazon-science/chronos-forecasting.git"

  Cloning https://github.com/amazon-science/chronos-forecasting.git to /tmp/pip-install-h20b8zfy/chronos-forecasting_aca7e98a093c4cd78e3d9f27b7541e5d
  Running command git clone --filter=blob:none --quiet https://github.com/amazon-science/chronos-forecasting.git /tmp/pip-install-h20b8zfy/chronos-forecasting_aca7e98a093c4cd78e3d9f27b7541e5d
  Resolved https://github.com/amazon-science/chronos-forecasting.git to commit 39515ff0fcdae55bbbf546d90193dbe54b201556
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for chronos-forecasting: filename=chronos_forecasting-1.4.1-py3-none-any.whl size=29154 sha256=0ddca36d3ab7c326aa669577503c52cb129ae029cab17162867aeeda02cdaa08
  Stored in directory: /tmp/pip-ephem-wheel-cache-rvvv__e7/wheels/ba/7f/47/c8e535215a6b02669af6db36392beef76752951c59bf8b5e74
Successfully built chronos-forecasting
  Attempting uninstall: chronos-forecasting
    Found existing

In [15]:
!ls -l output/run-0/checkpoint-final/

total 32820
-rw-r--r-- 1 root root     1196 Feb  1 17:11 config.json
-rw-r--r-- 1 root root      142 Feb  1 17:11 generation_config.json
-rw-r--r-- 1 root root 33588312 Feb  1 17:11 model.safetensors
-rw-r--r-- 1 root root     1839 Feb  1 17:11 training_info.json


In [17]:
from chronos import ChronosPipeline

pipeline = ChronosPipeline.from_pretrained("output/run-0/checkpoint-final/")